In [ ]:
!pip install cellpose

In [ ]:
import cellpose

In [ ]:
import shutil
import os
cell_dir_path = './cell'
if os.path.exists(cell_dir_path):
    shutil.rmtree(cell_dir_path)
shutil.copytree("../input/cell-data-20211207/cell_dataset", "./cell")

In [ ]:
!python -m cellpose \
--train \
--use_gpu \
--dir "/kaggle/working/cell/train" \
--test_dir "/kaggle/working/cell/test" \
--n_epochs 10 \
--learning_rate 0.002 \
--pretrained_model None

In [ ]:
!ls /kaggle/working/cell/train/models

In [ ]:
model_file = "/kaggle/working/cell/train/models/cellpose_residual_on_style_on_concatenation_off_train_2021_12_09_06_15_37.108116"

In [ ]:
import os
import numpy as np
import cv2
from scipy.ndimage import gaussian_filter
import scipy

global out_line_img

try:
    import matplotlib
    MATPLOTLIB_ENABLED = True 
except:
    MATPLOTLIB_ENABLED = False


try:
    from skimage import color
    from skimage.segmentation import find_boundaries
    SKIMAGE_ENABLED = True 
except:
    SKIMAGE_ENABLED = False


from cellpose import utils, io, transforms
from cellpose.omnipose.utils import ncolorlabel, sinebow

# modified to use sinebow color
def dx_to_circ(dP,transparency=False,mask=None):
    """ dP is 2 x Y x X => 'optic' flow representation 
    
    Parameters
    -------------
    
    dP: 2xLyxLx array
        Flow field components [dy,dx]
        
    transparency: bool, default False
        magnitude of flow controls opacity, not lightness (clear background)
        
    mask: 2D array 
        Multiplies each RGB component to suppress noise
    
    """
    
    dP = np.array(dP)
    mag = transforms.normalize99(np.sqrt(np.sum(dP**2,axis=0)),omni=1)
    angles = np.arctan2(dP[1], dP[0])+np.pi
    a = 2
    r = ((np.cos(angles)+1)/a)
    g = ((np.cos(angles+2*np.pi/3)+1)/a)
    b =((np.cos(angles+4*np.pi/3)+1)/a)
    
    if transparency:
        im = np.stack((r,g,b,mag),axis=-1)
    else:
        im = np.stack((r*mag,g*mag,b*mag),axis=-1)
        
    if mask is not None and transparency and dP.shape[0]<3:
        im[:,:,-1] *= mask
        
    return im


def show_segmentation(fig, img, maski, flowi, channels=[0,0], file_name=None, omni=False, seg_norm=False, bg_color=None):
    """ plot segmentation results (like on website)
    
    Can save each panel of figure with file_name option. Use channels option if
    img input is not an RGB image with 3 channels.
    
    Parameters
    -------------

    fig: matplotlib.pyplot.figure
        figure in which to make plot

    img: 2D or 3D array
        image input into cellpose

    maski: int, 2D array
        for image k, masks[k] output from Cellpose.eval, where 0=NO masks; 1,2,...=mask labels

    flowi: int, 2D array 
        for image k, flows[k][0] output from Cellpose.eval (RGB of flows)

    channels: list of int (optional, default [0,0])
        channels used to run Cellpose, no need to use if image is RGB

    file_name: str (optional, default None)
        file name of image, if file_name is not None, figure panels are saved
        
    omni: bool (optional, default False)
        use omni version of normalize99, image_to_rgb
        
    seg_norm: bool (optional, default False)
        improve cell visibility under labels
        
    bg_color: float (Optional, default none)
        background color to draw behind flow (visible if flow transparency is on)
        

    """
    if not MATPLOTLIB_ENABLED:
        raise ImportError("matplotlib not installed, install with 'pip install matplotlib'")
    ax = fig.add_subplot(1,4,1)
    img0 = img.copy()

    if img0.shape[0] < 4:
        img0 = np.transpose(img0, (1,2,0))
    if img0.shape[-1] < 3 or img0.ndim < 3:
        img0 = image_to_rgb(img0, channels=channels, omni=omni)
    else:
        if img0.max()<=50.0:
            img0 = np.uint8(np.clip(img0*255, 0, 1))
    
    ax.imshow(img0)
    ax.set_title('original image')
    ax.axis('off')

    outlines = utils.masks_to_outlines(maski)
    out_line_img = outlines
    c = sinebow(5)
    colors = np.array(list(c.values()))[1:] 
    
    # Image normalization to improve cell visibility under labels
    if seg_norm:
        fg = 1/9
        p = transforms.normalize99(img0,omni=omni)
        img1 = p**(np.log(fg)/np.log(np.mean(p[maski>0])))
    else:
        img1 = img0
    
    # the mask_overlay function changes colors (preserves only hue I think). The label2rgb function from
    # skimage.color works really well. 
    if SKIMAGE_ENABLED:
        overlay = color.label2rgb(ncolorlabel(maski),img1,colors,bg_label=0,alpha=1/3)
        overlay = np.uint8(np.clip(overlay, 0, 1)*255)
        overlay[maski==0] = img1[maski==0] #restore original level to background regions
    else:
        overlay = mask_overlay(img0, maski)

    ax = fig.add_subplot(1,4,2)
    outX, outY = np.nonzero(outlines)
    imgout= img0.copy()
    imgout[outX, outY] = np.array([255,0,0]) # pure red

    ax.imshow(imgout)
    ax.set_title('predicted outlines')
    ax.axis('off')

    ax = fig.add_subplot(1,4,3)
    ax.imshow(overlay)
    ax.set_title('predicted masks')
    ax.axis('off')

    ax = fig.add_subplot(1,4,4)
    if bg_color is not None:
        ax.imshow(np.ones_like(flowi)*bg_color)
    
    ax.imshow(flowi)
    ax.set_title('predicted cell pose')
    ax.axis('off')

    if file_name is not None:
        save_path = os.path.splitext(file_name)[0]
        io.imsave(save_path + '_overlay.jpg', overlay)
        io.imsave(save_path + '_outlines.jpg', imgout)
        io.imsave(save_path + '_flows.jpg', flowi)

def mask_rgb(masks, colors=None):
    """ masks in random rgb colors

    Parameters
    ----------------

    masks: int, 2D array
        masks where 0=NO masks; 1,2,...=mask labels

    colors: int, 2D array (optional, default None)
        size [nmasks x 3], each entry is a color in 0-255 range

    Returns
    ----------------

    RGB: uint8, 3D array
        array of masks overlaid on grayscale image

    """
    if colors is not None:
        if colors.max()>1:
            colors = np.float32(colors)
            colors /= 255
        colors = utils.rgb_to_hsv(colors)
    
    HSV = np.zeros((masks.shape[0], masks.shape[1], 3), np.float32)
    HSV[:,:,2] = 1.0
    for n in range(int(masks.max())):
        ipix = (masks==n+1).nonzero()
        if colors is None:
            HSV[ipix[0],ipix[1],0] = np.random.rand()
        else:
            HSV[ipix[0],ipix[1],0] = colors[n,0]
        HSV[ipix[0],ipix[1],1] = np.random.rand()*0.5+0.5
        HSV[ipix[0],ipix[1],2] = np.random.rand()*0.5+0.5
    RGB = (utils.hsv_to_rgb(HSV) * 255).astype(np.uint8)
    return RGB

def mask_overlay(img, masks, colors=None, omni=False):
    """ overlay masks on image (set image to grayscale)

    Parameters
    ----------------

    img: int or float, 2D or 3D array
        img is of size [Ly x Lx (x nchan)]

    masks: int, 2D array
        masks where 0=NO masks; 1,2,...=mask labels

    colors: int, 2D array (optional, default None)
        size [nmasks x 3], each entry is a color in 0-255 range

    Returns
    ----------------

    RGB: uint8, 3D array
        array of masks overlaid on grayscale image

    """
    if colors is not None:
        if colors.max()>1:
            colors = np.float32(colors)
            colors /= 255
        colors = utils.rgb_to_hsv(colors)
    if img.ndim>2:
        img = img.astype(np.float32).mean(axis=-1)
    else:
        img = img.astype(np.float32)
    
    
    HSV = np.zeros((img.shape[0], img.shape[1], 3), np.float32)
    HSV[:,:,2] = img
    hues = np.linspace(0, 1, masks.max()+1)
    for n in range(int(masks.max())):
        ipix = (masks==n+1).nonzero()
        if colors is None:
            HSV[ipix[0],ipix[1],0] = hues[n]
        else:
            HSV[ipix[0],ipix[1],0] = colors[n,0]
        HSV[ipix[0],ipix[1],1] = 1.0
    RGB = (utils.hsv_to_rgb(HSV) * 255).astype(np.uint8)
    return RGB

def image_to_rgb(img0, channels=[0,0], omni=False):
    """ image is 2 x Ly x Lx or Ly x Lx x 2 - change to RGB Ly x Lx x 3 """
    img = img0.copy()
    img = img.astype(np.float32)
    if img.ndim<3:
        img = img[:,:,np.newaxis]
    if img.shape[0]<5:
        img = np.transpose(img, (1,2,0))
    if channels[0]==0:
        img = img.mean(axis=-1)[:,:,np.newaxis]
    for i in range(img.shape[-1]):
        if np.ptp(img[:,:,i])>0:
            img[:,:,i] = transforms.normalize99(img[:,:,i],omni=omni)
            img[:,:,i] = np.clip(img[:,:,i], 0, 1)
    img *= 255
    img = np.uint8(img)
    RGB = np.zeros((img.shape[0], img.shape[1], 3), np.uint8)
    if img.shape[-1]==1:
        RGB = np.tile(img,(1,1,3))
    else:
        RGB[:,:,channels[0]-1] = img[:,:,0]
        if channels[1] > 0:
            RGB[:,:,channels[1]-1] = img[:,:,1]
    return RGB

def interesting_patch(mask, bsize=130):
    """ get patch of size bsize x bsize with most masks """
    Ly,Lx = mask.shape
    m = np.float32(mask>0)
    m = gaussian_filter(m, bsize/2)
    y,x = np.unravel_index(np.argmax(m), m.shape)
    ycent = max(bsize//2, min(y, Ly-bsize//2))
    xcent = max(bsize//2, min(x, Lx-bsize//2))
    patch = [np.arange(ycent-bsize//2, ycent+bsize//2, 1, int),
             np.arange(xcent-bsize//2, xcent+bsize//2, 1, int)]
    return patch

def disk(med, r, Ly, Lx):
    """ returns pixels of disk with radius r and center med """
    yy, xx = np.meshgrid(np.arange(0,Ly,1,int), np.arange(0,Lx,1,int),
                         indexing='ij')
    inds = ((yy-med[0])**2 + (xx-med[1])**2)**0.5 <= r
    y = yy[inds].flatten()
    x = xx[inds].flatten()
    return y,x

def outline_view(img0,maski,color=[1,0,0], mode='inner'):
    """
    Generates a red outline overlay onto image. 
    """
#     img0 = utils.rescale(img0)
    if len(img0.shape)<3:
#         img0 = image_to_rgb(img0) broken, transposing some images...
        img0 = np.stack([img0]*3,axis=-1)
    
    if SKIMAGE_ENABLED:
        outlines = find_boundaries(maski,mode=mode) #not using masks_to_outlines as that gives border 'outlines'
    else:
        outlines = utils.masks_to_outlines(maski,mode=mode) #not using masks_to_outlines as that gives border 'outlines'
    outY, outX = np.nonzero(outlines)
    imgout = img0.copy()
#     imgout[outY, outX] = np.array([255,0,0]) #pure red
    imgout[outY, outX] = np.array(color)

    return imgout

In [ ]:
import numpy as np
from cellpose import models, io, plot
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
global out_line_img
def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

test_dir = "../input/sartorius-cell-instance-segmentation/test/"
test_img_dirs = [test_dir + i for i in os.listdir(test_dir)]


test_imgs = []
for i in test_img_dirs:
    img = cv2.imread(i,cv2.IMREAD_COLOR)
    test_imgs.append(img)
# test_files = [fname for fname in test_dir.iterdir()]
# print(test_files)
# print(cv2.imread(str(test_files[0])).shape)
# plt.imshow(io.imread(str(test_files[0])))
# model = models.CellposeModel(gpu=True, pretrained_model='./cellpose_residual_on_style_on_concatenation_off_train_2021_12_08_07_16_13.734155')
# latest_file = "./cellpose_residual_on_style_on_concatenation_off_train_2021_12_08_07_16_13.734155"
model = models.CellposeModel(gpu=True, pretrained_model=model_file, torch=True, diam_mean=30.0, net_avg=True, device=None, residual_on=True, style_on=True, concatenation=False)


masks_all = []
styles_all = []
flows_all = []


for img in test_imgs:
    chan = [0, 1] # for black and white imgs
    #img = io.imread(filename)
    masks, flows, styles = model.eval(img, diameter=60, channels=chan)

    masks_all.append(masks)
    flows_all.append(flows)
    styles_all.append(styles)

# DISPLAY RESULTS

    fig = plt.figure(figsize=(12,5))
    show_segmentation(fig, img, masks, flows[0], channels=chan)
    plt.tight_layout()
    plt.show()


#model = models.Cellpose(gpu=False, model_type='cyto')
#model = models.Cellpose(gpu=True, model_type='cyto')




# ids, masks = [],[]
# for fn in test_files:
#     # preds, flows, _,_ = model.eval(io.imread(str(fn)), diameter=19, channels=[0,0], augment=True, resample=True)#when you run on your kernel.
#     preds, flows, _ = model.eval(cv2.imread(str(fn)), diameter=19, channels=[0,0], augment=True, resample=True)#when you submit.
    
#     plt.imshow(preds)
# #     print(preds.shape)
# #     break
    
#     for i in range (1, preds.max() + 1):
#         ids.append(fn.stem)
#         masks.append(rle_encode(preds == i))
        
# pd.DataFrame({'id':ids, 'predicted':masks}).to_csv('submission.csv', index=False)

In [ ]:
out_line_img

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,22))
plt.subplot(1,3,1)
plt.imshow(masks_all[0])
plt.subplot(1,3,2)
plt.imshow(masks_all[1])
plt.subplot(1,3,3)
plt.imshow(masks_all[2])